## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [122]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib qt


test_img = mpimg.imread('../test_images/test2.jpg') # Pick one test pic as example

plt.imshow(test_img)

## 1) Calibrate the Camera

In [218]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('../camera_cal/calibration*.jpg')


# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    nx = 9
    ny = 6
    ret, corners = cv2.findChessboardCorners(gray, (nx,ny),None) # 找到corners

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

In [177]:
def cal_undistort(img, objpoints, imgpoints):
    # Use cv2.calibrateCamera() and cv2.undistort()
    #undist = np.copy(img)  # Delete this line
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

test_undist = cal_undistort(test_img, objpoints, imgpoints)
plt.imshow(test_undist)

## 2) Create Binary Image

In [219]:
def binary_tf(img, s_thresh=(170, 255), sx_thresh=(100, 150)):
    img = np.copy(img)
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255
    
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
    
    return combined_binary

test_binary = binary_tf(test_undist, s_thresh=(170, 255), sx_thresh=(100, 150))
plt.imshow(test_binary)

## 4) Perspective Transform

In [220]:
def perspective_tf(img):
    src = np.float32([[250,705],[1155,705],[785,475],[530,475]])
    dst = np.float32([[250,705],[1155,705],[1155,455],[250,455]])   
    M = cv2.getPerspectiveTransform(src, dst)
    img_size = (img.shape[1],img.shape[0])
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)    
    return warped,M

test_persp,M = perspective_tf(test_binary)
plt.imshow(test_persp)


## 5) Find Lane Boundary

In [221]:
def find_lane_pixels(test_persp):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(test_persp[test_persp.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((test_persp, test_persp, test_persp))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(test_persp.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = test_persp.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = test_persp.shape[0] - (window+1)*window_height
        win_y_high = test_persp.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def fit_polynomial(test_persp):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(test_persp)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)  # 拟合曲线的系数
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, test_persp.shape[0]-1, test_persp.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    # Plots the left and right polynomials on the lane lines
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')

    return out_img,left_fitx, right_fitx, ploty # left_fitx和right_fitx是拟合出的曲线


test_boundline,left_fitx, right_fitx,ploty = fit_polynomial(test_persp)

plt.imshow(test_boundline)

## 6) Calculate the Curvature

In [222]:
def process_data(leftx, rightx,test_persp):
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/875 # meters per pixel in x dimension
    # Fit a second order polynomial to pixel positions in each fake lane line
    ##### TO-DO: Fit new polynomials to x,y in world space #####
    ##### Utilize `ym_per_pix` & `xm_per_pix` here #####
    ploty_left = np.linspace(0, test_persp.shape[0]-1, len(leftx) )*ym_per_pix
    ploty_right = np.linspace(0, test_persp.shape[0]-1, len(rightx) )*ym_per_pix
    leftx = leftx*xm_per_pix
    rightx = rightx*xm_per_pix
    left_fit_cr = np.polyfit(ploty_left, leftx, 2)
    right_fit_cr = np.polyfit(ploty_right, rightx, 2)
    
    return  ploty_left,ploty_right, left_fit_cr, right_fit_cr

    
def measure_curvature_real(ploty_left,ploty_right, left_fit_cr, right_fit_cr):
    left_curverad = ((1 + (2*left_fit_cr[0]*np.max(ploty_left) + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])  ## Implement the calculation of the left line here
    right_curverad = ((1 + (2*right_fit_cr[0]*np.max(ploty_right) + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0]) ## Implement the calculation of the right line here
    return left_curverad, right_curverad
# Calculate the radius of curvature in meters for both lane lines

leftx, lefty, rightx, righty, out_img = find_lane_pixels(test_persp)
ploty_left,ploty_right, left_fit_cr, right_fit_cr = process_data(leftx, rightx, test_persp)
left_curverad, right_curverad = measure_curvature_real(ploty_left,ploty_right, left_fit_cr, right_fit_cr)

print(left_curverad, 'm', right_curverad, 'm')



427.03932402099286 m 714.1531152313975 m


### Question: <font color=coral >why leftx and rightx are different length? -> lead ploty should be two length </font>

## 7) Plot the lane on the original picture

In [236]:
def plot_area(test_undist,test_persp,M):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(test_persp).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    #plt.imshow(warp_zero)

    # Recast the x and y points into usable format for cv2.fillPoly()
    ym_per_pix = 30/720
   # ploty= np.linspace(0, test_undist.shape[0]-1, test_indist.shape[0] )*ym_per_pix
    test_boundline,left_fitx, right_fitx,ploty = fit_polynomial(test_persp)
   # print(left_fitx)
   # print(ploty)
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255,0))
    #plt.imshow(color_warp)
    Minv = np.mat(M).I
    newwarp = cv2.warpPerspective(color_warp, Minv,(test_undist.shape[1], test_undist.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(test_undist, 1, newwarp, 0.3, 0)
    plt.imshow(result)

In [237]:
#plt.imshow(test_undist)
plot_area(test_undist,test_persp,M)